In [23]:
import pandas as pd
import csv 
import sqlite3
from geopy.geocoders import OpenCage
import requests
import re

In [2]:
nobel_data = "Resources/nobel_latest.csv"

In [41]:
nobel_df = pd.read_csv(nobel_data)
nobel_df

,Year,Laureate_Id,Firstname,Lastname,Category,Gender,Prize_Share,Motivation,Birth_Date,Birth_Country,Birth_City,Birth_Country_Code,Death_Date,Death_Country,Death_City,Death_Country_Code,Organization_Name,Organization_City,Organization_Country
0,1901,1,Wilhelm Conrad,Röntgen,physics,male,1,"""in recognition of the extraordinary services ...",1845-03-27,Prussia (now Germany),Lennep (now Remscheid),DE,10/2/1923,Germany,Munich,DE,Munich University,Munich,Germany
1,1902,2,Hendrik A.,Lorentz,physics,male,2,"""in recognition of the extraordinary service t...",1853-07-18,the Netherlands,Arnhem,NL,4/2/1928,the Netherlands,NaN,NL,Leiden University,Leiden,the Netherlands
2,1902,3,Pieter,Zeeman,physics,male,2,"""in recognition of the extraordinary service t...",1865-05-25,the Netherlands,Zonnemaire,NL,9/10/1943,the Netherlands,Amsterdam,NL,Amsterdam University,Amsterdam,the Netherlands
3,1903,4,Henri,Becquerel,physics,male,2,"""in recognition of the extraordinary services ...",1852-12-15,France,Paris,FR,25-08-1908,France,NaN,FR,École Polytechnique,Paris,France
4,1903,5,Pierre,Curie,physics,male,4,"""in recognition of the extraordinary services ...",1859-05-15,France,Paris,FR,19-04-1906,France,Paris,FR,École municipale de physique et de chimie indu...,Paris,France
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
984,2022,1019,Memorial,NaN,peace,org,3,"""The Peace Prize laureates represent civil soc...",1987-00-00,NaN,NaN,NaN,0000-00-00,NaN,NaN,NaN,NaN,NaN,NaN
985,2022,1020,Center for Civil Liberties,NaN,peace,org,3,"""The Peace Prize laureates represent civil soc...",2007-00-00,NaN,NaN,NaN,0000-00-00,NaN,NaN,NaN,NaN,NaN,NaN
986,2022,1021,Ben,Bernanke,economics,male,3,"""for research on banks and financial crises""",13-12-1953,USA,Augusta GA,US,0000-00-00,NaN,NaN,NaN,The Brookings Institution,Washington D.C.,USA
987,2022,1022,Douglas,Diamond,economics,male,3,"""for research on banks and financial crises""",25-10-1953,USA,Chicago IL,US,0000-00-00,NaN,NaN,NaN,University of Chicago,Chicago IL,USA


In [66]:
cleaned_nobel_df = nobel_df.drop(["Prize_Share", "Birth_City", "Birth_Country_Code", "Death_Country", "Death_City", "Death_Country_Code",
               "Organization_Name", "Organization_City", "Organization_Country"], axis = 1)

In [67]:
cleaned_nobel_df['Year'] = cleaned_nobel_df['Year'].astype('Int64')
cleaned_nobel_df['Laureate_Id'] = cleaned_nobel_df['Laureate_Id'].astype(str)
cleaned_nobel_df["Name"] = cleaned_nobel_df["Firstname"].astype(str)+ " " + cleaned_nobel_df["Lastname"].astype(str)

In [68]:
column_to_move = cleaned_nobel_df.pop("Name")

cleaned_nobel_df.insert(2, "Name", column_to_move)

In [69]:
full_name_df = cleaned_nobel_df.drop(["Firstname", "Lastname"], axis= 1)

In [70]:
full_name_df.head(10)

,Year,Laureate_Id,Name,Category,Gender,Motivation,Birth_Date,Birth_Country,Death_Date
0,1901,1,Wilhelm Conrad Röntgen,physics,male,"""in recognition of the extraordinary services ...",1845-03-27,Prussia (now Germany),10/2/1923
1,1902,2,Hendrik A. Lorentz,physics,male,"""in recognition of the extraordinary service t...",1853-07-18,the Netherlands,4/2/1928
2,1902,3,Pieter Zeeman,physics,male,"""in recognition of the extraordinary service t...",1865-05-25,the Netherlands,9/10/1943
3,1903,4,Henri Becquerel,physics,male,"""in recognition of the extraordinary services ...",1852-12-15,France,25-08-1908
4,1903,5,Pierre Curie,physics,male,"""in recognition of the extraordinary services ...",1859-05-15,France,19-04-1906
5,1903,6,Marie Curie,physics,female,"""in recognition of the extraordinary services ...",1867-11-07,Russian Empire (now Poland),4/7/1934
6,1911,6,Marie Curie,chemistry,female,"""in recognition of her services to the advance...",1867-11-07,Russian Empire (now Poland),4/7/1934
7,1904,8,Lord Rayleigh,physics,male,"""for his investigations of the densities of th...",1842-11-12,United Kingdom,30-06-1919
8,1905,9,Philipp Lenard,physics,male,"""for his work on cathode rays""",1862-06-07,Hungary (now Slovakia),20-05-1947
9,1906,10,J.J. Thomson,physics,male,"""in recognition of the great merits of his the...",1856-12-18,United Kingdom,30-08-1940


In [72]:
full_name_df["Birth_Country"] = full_name_df["Birth_Country"].astype(str)

In [74]:
def clean_country_name(place):
    # Find the text inside parentheses
    match = re.search(r'\(([^()]+)\)', place)
    
    if match:
        # Split the text inside parentheses and extract the last word
        words = match.group(1).split()
        if len(words) >= 2:
            country_name = words[-1]
        else:
            country_name = place
    else:
        country_name = place
    
    return country_name

# Apply the function to clean the column
full_name_df['Birth_Country'] = full_name_df['Birth_Country'].apply(clean_country_name)

In [75]:
full_name_df.head(10)

,Year,Laureate_Id,Name,Category,Gender,Motivation,Birth_Date,Birth_Country,Death_Date
0,1901,1,Wilhelm Conrad Röntgen,physics,male,"""in recognition of the extraordinary services ...",1845-03-27,Germany,10/2/1923
1,1902,2,Hendrik A. Lorentz,physics,male,"""in recognition of the extraordinary service t...",1853-07-18,the Netherlands,4/2/1928
2,1902,3,Pieter Zeeman,physics,male,"""in recognition of the extraordinary service t...",1865-05-25,the Netherlands,9/10/1943
3,1903,4,Henri Becquerel,physics,male,"""in recognition of the extraordinary services ...",1852-12-15,France,25-08-1908
4,1903,5,Pierre Curie,physics,male,"""in recognition of the extraordinary services ...",1859-05-15,France,19-04-1906
5,1903,6,Marie Curie,physics,female,"""in recognition of the extraordinary services ...",1867-11-07,Poland,4/7/1934
6,1911,6,Marie Curie,chemistry,female,"""in recognition of her services to the advance...",1867-11-07,Poland,4/7/1934
7,1904,8,Lord Rayleigh,physics,male,"""for his investigations of the densities of th...",1842-11-12,United Kingdom,30-06-1919
8,1905,9,Philipp Lenard,physics,male,"""for his work on cathode rays""",1862-06-07,Slovakia,20-05-1947
9,1906,10,J.J. Thomson,physics,male,"""in recognition of the great merits of his the...",1856-12-18,United Kingdom,30-08-1940


In [76]:
def get_coordinates_from_wikipedia(page_title):
    base_url = 'https://en.wikipedia.org/w/api.php'
    params = {
        'action': 'query',
        'format': 'json',
        'prop': 'coordinates|info',
        'titles': page_title
    }

    response = requests.get(base_url, params=params)
    data = response.json()
    # Extract the coordinates and URL from the response
    pages = data['query']['pages']
    page_id = next(iter(pages))
    page_data = pages[page_id]
    coordinates = page_data.get('coordinates', [])
    url = page_data.get('fullurl')

    latitude = coordinates[0]['lat'] if coordinates else None
    longitude = coordinates[0]['lon'] if coordinates else None

    return latitude, longitude, url

# Initialize empty lists for coordinates and URLs
coordinates = []
wiki_urls = []

# Loop over the names in the DataFrame to retrieve coordinates
for country in full_name_df['Birth_Country']:
    page_title = country.replace(' ', '_')
    page_title += ' (now)'

    latitude, longitude, _ = get_coordinates_from_wikipedia(page_title)
    coordinates.append((latitude, longitude))

# Loop over the names in the DataFrame to retrieve URLs
for name in full_name_df['Name']:
    page_title = name.replace(' ', '_')
    page_title += '_(Died)'

    _, _, url = get_coordinates_from_wikipedia(page_title)
    wiki_urls.append(url)

# Add the coordinates and URLs to the existing DataFrame
full_name_df['Coordinates'] = coordinates
full_name_df['Wikipedia_URL'] = wiki_urls

# Print the updated DataFrame
print(full_name_df)

     Year Laureate_Id                            Name   Category Gender  \
0    1901           1          Wilhelm Conrad Röntgen    physics   male   
1    1902           2              Hendrik A. Lorentz    physics   male   
2    1902           3                   Pieter Zeeman    physics   male   
3    1903           4                 Henri Becquerel    physics   male   
4    1903           5                    Pierre Curie    physics   male   
..    ...         ...                             ...        ...    ...   
984  2022        1019                    Memorial nan      peace    org   
985  2022        1020  Center for Civil Liberties nan      peace    org   
986  2022        1021                    Ben Bernanke  economics   male   
987  2022        1022                 Douglas Diamond  economics   male   
988  2022        1023                   Philip Dybvig  economics   male   

                                            Motivation  Birth_Date  \
0    "in recognition of the e

In [36]:
con = sqlite3.connect("data/nobel_prize.db")
curs = con.cursor()

curs.execute("DROP TABLE IF EXISTS nobel_winners")

curs.execute("""CREATE TABLE recipients (
    Year INTEGER,
    id INTEGER PRIMARY KEY,
    Laureate_Id,
    Name TEXT,
    Category TEXT,
    Gender TEXT,
    Motivation TEXT,
    Birth_Date TEXT,
    Birth_Country TEXT,
    Death_Date TEXT
    )
""")
full_name_df.to_sql("recipients", con, if_exists="append", index=False)

con.commit()
con.close()

